Q1:
The environment is limited, deterministic, and the states are completely observable.

Q2:
The agents are:
qlearner - performs actions based on the qlearning equation
guru - performs the best action mathematically
random - makes actions randomly

Q3:
The reward is a positive number that is used to update the learning table in case the learner wins. The penalty is negative and is for losses.

In [1]:
states = set({})
def calculate_states(current,total_actions,add_one):
    if current in states:
        return
    states.add(current)
    if add_one:
        total_actions[0]+=1
    for i in range(3):
        for j in range(1,current[0][i]):
            n = list(current[0])
            n[i]-=j
            n = sorted(n)#
            calculate_states((tuple(n),1-current[1]),total_actions, not add_one)

total_actions = [0]
calculate_states( ((10,10,10),0),total_actions, False )
print(len(states))
print(total_actions)

436
[218]


Q4:11^3 is 1,331.

Q5:Sum of the first 30 numbers is 465.

In [2]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [3]:
def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

In [4]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [5]:
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  505    Random  495
1000 games,     Guru 1000    Random    0
1000 games,   Random    6      Guru  994
1000 games,     Guru  949      Guru   51


In [6]:
qtable, Alpha, Gamma, Reward = None, 1, 0.8, 100.0#change alpha from 1 to #change gamma from .8 to
# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(-Reward, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            #penalize losses ^
            # Switch sides for play and learning
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [7]:
%%time

nim_qlearn(1000)

Wall time: 109 ms


In [8]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')

play_games(1000, 'Random', 'Random') ;

1000 games, Qlearner  762    Random  238
1000 games,   Random  241  Qlearner  759
1000 games,   Random  479    Random  521


In [9]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner  541    Random  459
1000 games, Qlearner  593    Random  407
1000 games, Qlearner  689    Random  311
1000 games, Qlearner  771    Random  229
1000 games, Qlearner  784    Random  216
1000 games, Qlearner  775    Random  225
1000 games, Qlearner  782    Random  218
Wall time: 13.6 s


In [10]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(Wins)

[0.541, 0.593, 0.689, 0.771, 0.784, 0.775, 0.782]


In [11]:
# Function to print the entire set of states
def qtable_log(_fn:str):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtable_debug.txt')

In [12]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Guru')
    Wins += [wins_a/(wins_a+wins_b)]
# Check the ratio of wins wrt to size of the reinforcement model training
print(Wins)

1000 games, Qlearner    5      Guru  995
1000 games, Qlearner    2      Guru  998
1000 games, Qlearner    3      Guru  997
1000 games, Qlearner   10      Guru  990
1000 games, Qlearner   16      Guru  984
1000 games, Qlearner   11      Guru  989
1000 games, Qlearner   20      Guru  980
[0.005, 0.002, 0.003, 0.01, 0.016, 0.011, 0.02]
Wall time: 14.1 s


Q6:
By simply changing the penalty for a loss from 0 to `-Reward`, we get improved performance in the QLearner.  I'm not sure how to beat the Guru.